In [1]:
from ltp import LTP
ltp = LTP() # 默认加载 Small 模型path='D:\\webdownload\\small1.tgz'

In [2]:
ltp.add_words(words=["人大代表", "生态文明","扬子江","长江大保护","习近平",'禁捕','退捕','禁渔'], max_window=5)

In [1]:
!pip install ltp==4.1.3

  Using cached ltp-4.1.3-py3-none-any.whl (74 kB)
  Using cached transformers-4.4.2-py3-none-any.whl (2.0 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 3.4.0
    Uninstalling transformers-3.4.0:
      Successfully uninstalled transformers-3.4.0
  Attempting uninstall: ltp
    Found existing installation: ltp 4.0.6
    Uninstalling ltp-4.0.6:
      Successfully uninstalled ltp-4.0.6


ERROR: -tp 4.0.10 has requirement transformers<=3.4.0,>=3.0, but you'll have transformers 4.4.2 which is incompatible.


In [1]:
!pip install torch==1.4.0 torchvision==0.5.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu92/torchvision-0.5.0%2Bcu92-cp38-cp38-win_amd64.whl (1.2 MB)


## semantic character model

In [1]:
##a0-v-a1 equals to Sn v Gn
##v-a1 equals to v Gn
##a1-v equals to Gn v

In [3]:
import pandas as pd
#data=pd.read_csv('C:\\Users\\10312\\Desktop\\changjiang.csv')
data=pd.read_csv('data.csv')
data=data.sample(n=4000,random_state=1,axis=0)
a=data['摘要'].drop_duplicates().values
a=a.tolist()

In [4]:
len(a)

255

In [5]:
def getA(seg,i,j):
    s=''
    if i==j:
        s=seg[0][i]
    else:
        while i<=j:
            s=s+seg[0][i]
            i=i+1
    return s

In [8]:
def matchw(zz,dict1,dict2,dict3):
    for k in zz:
            k=k.replace(' ','')
            k=k.replace('[','（')
            k=k.replace(']','）')
            k=k.replace('(','（')
            k=k.replace(')','）')
            k=k.replace('^','.')
            try:
                seg,hidden=ltp.seg([k])
            except Exception:
                print(k)
                print(zz)
            srl = ltp.srl(hidden, keep_empty=False)
            tt=0                
            start=0
            end=0
            e=0
            for i in range(len(srl[0])-1,-1,-1):
                f1=0
                f2=0
                b=''
                c=''
                d=''
                if len(seg[0][srl[0][i][0]])>1:
                    for j in srl[0][i][1]:
                        #print(j)
                        if tt==0:
                            if j[0]=='A0':
                                a1=getA(seg,j[1],j[2])
                                f1=1
                                end=j[2]
                            elif j[0]=='A1':
                                b=getA(seg,j[1],j[2])
                                if j[1]>srl[0][i][0]:
                                    c=b
                                elif j[1]<srl[0][i][0]:
                                    d=b
                                start=j[1]
                                f2=1
                                tt=1
                            elif j[0]=='A2':
                                if b=='':
                                    b=getA(seg,j[1],j[2])
                                    if j[1]>srl[0][i][0]:
                                        c=b
                                    elif j[1]<srl[0][i][0]:
                                        d=b
                                start=j[1]
                                f2=1
                                tt=1
                        else:
                            if j[0]=='A0':
                                a1=getA(seg,j[1],j[2])
                                f1=1
                                end=j[2]
                            elif j[0]=='A1' and j[2]<start :
                                b=getA(seg,j[1],j[2])
                                #print(b)
                                if j[1]>srl[0][i][0]:
                                    c=b
                                elif j[1]<srl[0][i][0]:
                                    d=b
                                    #print(d)
                                start=j[1]
                                f2=1
                            elif j[0]=='A2' and j[2]<start :
                                if b=='':
                                    b=getA(seg,j[1],j[2])
                                    if j[1]>srl[0][i][0]:
                                        c=b
                                    elif j[1]<srl[0][i][0]:
                                        d=b
                                start=j[1]
                                f2=1
                    #print('start')
                    #print(start)
                    #print('end')
                    #print(end)
                    #print('start')
                    #print(start)
                    #print('end')
                    #print(end)
                    if f1==1 and f2==1 and end<=start and len(seg[0][srl[0][i][0]])==2:
                        if (a1,seg[0][srl[0][i][0]],b) not in dict1.keys():
                            dict1[(a1,seg[0][srl[0][i][0]],b)]=1
                        else:
                            dict1[(a1,seg[0][srl[0][i][0]],b)]=dict1[(a1,seg[0][srl[0][i][0]],b)]+1
                    elif f1==0 and f2==1 and c!='' and len(seg[0][srl[0][i][0]])==2:
                        if (seg[0][srl[0][i][0]],c) not in dict2.keys():
                            dict2[(seg[0][srl[0][i][0]],c)]=1
                        else:
                            dict2[(seg[0][srl[0][i][0]],c)]=dict2[(seg[0][srl[0][i][0]],c)]+1
                    elif f1==0 and f2==1 and d!='' and len(seg[0][srl[0][i][0]])==2:
                        if (seg[0][srl[0][i][0]],d) not in dict3.keys():
                            dict3[(seg[0][srl[0][i][0]],d)]=1
                        else:
                            dict3[(seg[0][srl[0][i][0]],d)]=dict3[(seg[0][srl[0][i][0]],d)]+1

In [9]:
dict1={}
dict2={}
dict3={}
for z in a:
    z=z.replace('\u200b','')
    z=z.replace('.','^')
    zz=ltp.sent_split([z])
    matchw(zz,dict1,dict2,dict3)

In [10]:
data=pd.DataFrame()

In [12]:
import re
#s="2019“长江流域重点水域禁捕和建立补偿制度实施方案”的颁布施行"
#mat=re.search( r'《(.*)》', s)
#mat.group()

In [13]:
# a0-v-a1
a0=[]
v=[]
a1=[]
n=[]
for i in dict1.keys():
    if len(i[0])>1 and len(i[1])==2:
        mat=re.search( r'《(.*)》', i[0])
        nat=re.search(r'“(.*)”',i[0])
        a=""
        if mat:
            a=a+mat.group()
        
        if nat:
            a=a+nat.group()
        if len(a)==0:
            a0.append(i[0])
        else:
            a0.append(a)
        v.append(i[1])
        a1.append(i[2])
        n.append(dict1[i])
for i in [a0,v,a1,n]:
    df=pd.DataFrame((v for v in i))
    data=pd.concat([data,df],axis=1)
data.columns=['a0','v','a1','count']

In [14]:
#a0-v-a1
data.to_csv('a0_v_a1.csv',index=False,encoding='utf_8_sig')

In [15]:
data=pd.DataFrame()

In [16]:
#v-a1
v=[]
a1=[]
n=[]
for i in dict2.keys():
    v.append(i[0])
    a1.append(i[1])
    n.append(dict2[i])
for i in [v,a1,n]:
    df=pd.DataFrame((v for v in i))
    data=pd.concat([data,df],axis=1)
data.columns=['v','a1','count']

In [17]:
#v-a1
data.to_csv('v_a1.csv',index=False,encoding='utf_8_sig')

In [18]:
data=pd.DataFrame()

In [19]:
#a1-v
v=[]
a1=[]
n=[]
for i in dict3.keys():
    v.append(i[0])
    a1.append(i[1])
    n.append(dict3[i])
for i in [a1,v,n]:
    df=pd.DataFrame((v for v in i))
    data=pd.concat([data,df],axis=1)
data.columns=['a1','v','count']

In [20]:
#a1-v
data.to_csv('a1-v.csv',index=False,encoding='utf_8_sig')

In [102]:
#data=pd.DataFrame()

In [ ]:
# a0-v-a1
#a0=[]
#v=[]
#a1=[]
#n=[]
#for i in dict1.keys():
#    a0.append(i[0])
#    v.append(i[1])
#    a1.append(i[2])
#    n.append(dict1[i])
#for i in [a0,v,a1,n]:
#    df=pd.DataFrame((v for v in i))
#    data=pd.concat([data,df],axis=1)
#data.columns=['a0','v','a1','count']

In [104]:
#data['count']=data['count']+1

In [67]:
#a0-v-a1
#data.to_csv('C:\\Users\\10312\\Desktop\\a0_v_a1_new.csv',index=False,encoding='utf_8_sig')